## Obtaining, cleaning and preparing data

In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
%matplotlib notebook

In [2]:
#datetime data range

from_date = datetime.date(1900,1,1)
to_date = datetime.date(2021,11,1)

In [13]:
#obtaining data from Federal Reserve Economic Data | FRED | St. Louis Fed

fred_data = web.DataReader(['FEDFUNDS','GS10','M2NS','PERMIT','UMCSENT','TCU','UMTMNO',
                            'PAYNSA','UNRATE','RSXFSN','CPILFENS'],
                               data_source='fred',start=from_date, end=to_date)
fred_data.dropna(inplace=True)

In [16]:
fred_data.head()

,FEDFUNDS,GS10,M2NS,PERMIT,UMCSENT,TCU,UMTMNO,PAYNSA,UNRATE,RSXFSN,CPILFENS
DATE,,,,,,,,,,,
1992-02-01,4.06,7.34,3390.1,1146.0,68.8,79.8145,228230.0,106977,7.4,131244.0,145.6
1992-03-01,3.98,7.54,3404.8,1082.0,76.0,80.3033,245052.0,107495,7.4,142488.0,146.4
1992-04-01,3.73,7.48,3418.9,1054.0,77.2,80.7450,236726.0,108330,7.4,147175.0,146.6
1992-05-01,3.82,7.39,3388.0,1056.0,79.2,80.8311,238178.0,109137,7.6,152420.0,146.7
1992-06-01,3.76,7.26,3388.7,1057.0,80.4,80.6997,254919.0,109626,7.8,151849.0,146.9


In [28]:
# changing working directory

import os

cwd = os.getcwd()
os.chdir('D:/Data Science Projects/US_Core_Inflation_Forecast')
print("Current working directory: {0}".format(cwd))

# obtaining data from csv file (Institute for Supply and Management PMI)

ism_man_data = pd.read_csv('ISM Manufacturing.csv')
ism_serv_data = pd.read_csv('ISM Services.csv')

Current working directory: D:\Data Science Projects\US_Core_Inflation_Forecast


In [39]:
print(ism_man_data.head())
print(ism_serv_data.head())

         Release Date Observation Date  PMI M Composite  PMI M Prices
0  Jan 01, 1970 (Dec)           Dec-69             52.0          82.2
1  Feb 01, 1970 (Jan)           Jan-70             48.7          80.6
2  Mar 01, 1970 (Feb)           Feb-70             47.4          74.5
3  Apr 01, 1970 (Mar)           Mar-70             46.9          67.8
4  May 01, 1970 (Apr)           Apr-70             45.0          75.1
         Release Date Observation Date  PMI S Composite  PMI S Prices
0  Aug 01, 1997 (Jul)           Jul-97             56.7          50.9
1  Sep 01, 1997 (Aug)           Aug-97             62.0          53.3
2  Oct 01, 1997 (Sep)           Sep-97             56.2          53.1
3  Nov 01, 1997 (Oct)           Oct-97             56.6          53.1
4  Dec 01, 1997 (Nov)           Nov-97             58.5          54.3


In [30]:
# cleaning, sorting and merging databases

    #ISM databases
ism_data = pd.merge(left=ism_serv_data, right=ism_man_data, how='inner', on='Observation Date')
ism_data.drop(columns=['Release Date_x','Release Date_y'],inplace=True)
ism_data['Observation Date'] = pd.to_datetime(arg=ism_data['Observation Date'], format='%b-%y', infer_datetime_format=True)
ism_data.set_index('Observation Date', inplace=True)

    #FRED databases
fred_data.columns = ['FED Funds Rate','10 Year Treasury Rate','M2','Building Permits',
                    'Consumer Sentiment','Capacity Utilization','Manufacturing New Orders',
                     'Nonfarm Employment','Unemployment Rate SA','Retail Trade','CPI Core NSA']

In [31]:
# checking data types

print(fred_data.dtypes)
print(ism_data.dtypes)

FED Funds Rate              float64
10 Year Treasury Rate       float64
M2                          float64
Building Permits            float64
Consumer Sentiment          float64
Capacity Utilization        float64
Manufacturing New Orders    float64
Nonfarm Employment            int64
Unemployment Rate SA        float64
Retail Trade                float64
CPI Core NSA                float64
dtype: object
PMI S Composite    float64
PMI S Prices       float64
PMI M Composite    float64
PMI M Prices       float64
dtype: object


In [40]:
# merging ism and fred data

data_all = pd.merge(ism_data,fred_data,how='inner',left_index=True, right_index=True)
data_all.tail()

,PMI S Composite,PMI S Prices,PMI M Composite,PMI M Prices,FED Funds Rate,10 Year Treasury Rate,M2,Building Permits,Consumer Sentiment,Capacity Utilization,Manufacturing New Orders,Nonfarm Employment,Unemployment Rate SA,Retail Trade,CPI Core NSA
2021-07-01,64.1,82.3,59.5,85.7,0.10,1.32,20609.1,1630.0,81.2,76.2126,492093.0,146536,5.4,558031.0,279.146
2021-08-01,61.7,75.4,59.9,79.4,0.09,1.28,20810.7,1721.0,70.3,76.0702,525042.0,147012,5.2,557018.0,279.507
2021-09-01,61.9,77.5,61.1,81.2,0.08,1.37,20995.6,1586.0,72.8,75.2589,529459.0,147650,4.7,536229.0,279.884
2021-10-01,66.7,82.9,60.8,85.7,0.08,1.58,21178.6,1653.0,71.7,76.4801,524174.0,149245,4.6,559301.0,281.617
2021-11-01,69.1,82.3,61.1,82.4,0.08,1.56,21425.9,1717.0,67.4,76.8314,517146.0,150098,4.2,579368.0,282.754


In [48]:
# data manipulation

data_all['Released Date'] = data_all.index + pd.DateOffset(months=1) # release date for economic indicators (1 month lag)
data_all['PMI Composite Avg'] = np.average(data_all[['PMI S Composite','PMI M Composite']],axis=1) # PMI Composite Average
data_all['PMI Prices Avg'] = np.average(data_all[['PMI S Prices','PMI M Prices']],axis=1) # PMI Prices Average
                                
data_all = data_all[['Released Date',
                     'PMI M Composite','PMI S Composite','PMI Composite Avg',
                     'PMI M Prices','PMI S Prices','PMI Prices Avg',
                     'Building Permits','Consumer Sentiment',
                     'FED Funds Rate','10 Year Treasury Rate','M2',
                     'Capacity Utilization','Manufacturing New Orders',
                     'Nonfarm Employment','Unemployment Rate SA',
                     'Retail Trade',
                     'CPI Core NSA']]

# percentage change calculation

data_calc = data_all.copy(deep=True)
data_calc['Building Permits YoY'] = data_calc['Building Permits'].pct_change(periods=12) * 100
data_calc['Consumer Sentiment YoY'] = data_calc['Consumer Sentiment'].pct_change(periods=12) * 100
data_calc['M2 YoY'] = data_calc['M2'].pct_change(periods=12) * 100
data_calc['Capacity Utilization YoY'] = data_calc['Capacity Utilization'].pct_change(periods=12)
data_calc['Manufacturing New Orders YoY'] = data_calc['Manufacturing New Orders'].pct_change(periods=12) * 100
data_calc['Nonfarm Employment YoY'] = data_calc['Nonfarm Employment'].pct_change(periods=12) * 100
data_calc['Retail Trade YoY'] = data_calc['Retail Trade'].pct_change(periods=12) * 100
data_calc['CPI Core NSA YoY'] = data_calc['CPI Core NSA'].pct_change(periods=12) * 100

In [49]:
data_calc.head(15)

,Released Date,PMI M Composite,PMI S Composite,PMI Composite Avg,PMI M Prices,PMI S Prices,PMI Prices Avg,Building Permits,Consumer Sentiment,FED Funds Rate,...,Retail Trade,CPI Core NSA,Building Permits YoY,Consumer Sentiment YoY,M2 YoY,Capacity Utilization YoY,Manufacturing New Orders YoY,Nonfarm Employment YoY,Retail Trade YoY,CPI Core NSA YoY
1997-07-01,1997-08-01,57.7,56.7,57.20,52.0,50.9,51.45,1440.0,107.1,5.52,...,208856.0,169.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-08-01,1997-09-01,56.3,62.0,59.15,52.1,53.3,52.70,1449.0,104.4,5.54,...,211952.0,169.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-09-01,1997-10-01,53.9,56.2,55.05,53.0,53.1,53.05,1494.0,106.0,5.54,...,201415.0,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-01,1997-11-01,56.4,56.6,56.50,53.6,53.1,53.35,1499.0,105.6,5.50,...,210145.0,170.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-11-01,1997-12-01,55.7,58.5,57.10,52.1,54.3,53.20,1469.0,107.2,5.52,...,207533.0,170.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-12-01,1998-01-01,54.5,55.5,55.00,52.2,54.9,53.55,1456.0,102.1,5.50,...,249389.0,170.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-01,1998-02-01,53.8,57.0,55.40,47.0,52.7,49.85,1555.0,106.6,5.56,...,187445.0,171.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-01,1998-03-01,52.9,56.2,54.55,45.5,51.6,48.55,1647.0,110.4,5.51,...,183767.0,172.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-01,1998-04-01,52.9,54.7,53.80,44.2,47.9,46.05,1605.0,106.5,5.49,...,207973.0,172.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-04-01,1998-05-01,52.2,54.9,53.55,40.5,46.8,43.65,1547.0,108.7,5.45,...,212262.0,173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ---------- stopped here -----------

In [46]:
# time shifting and correlation sweet spot

data_shift = data_calc[12:]
data_shift = data_shift[['Index M','Index S','PMI Average','Building Permits','Consumer Sentiment','FED Funds Rate',
                        '10 Year Treasury Rate','Building Permits YoY','Consumer Sentiment YoY','M2 YoY',
                        'CPI Core NSA YoY']]

time_shift = list(range(0,13))
columns_x = ['Index M','Index S','PMI Average','Building Permits','Consumer Sentiment','FED Funds Rate',
             '10 Year Treasury Rate','Building Permits YoY','Consumer Sentiment YoY','M2 YoY']
column_y = ['CPI Core NSA YoY']
corr_results = {} # key:columns, values:corr

for x in columns_x:
    d = data_shift[[x,'CPI Core NSA YoY']]
    corr_results.setdefault(x,[])
    for t in time_shift:
        pearson = d[x].shift(t).corr(d['CPI Core NSA YoY'],method='pearson')
        corr_results[x].append(pearson)
        
def corr_sweet_spot(columns): # columns must be iterable (list)
    
    time_shift = {}
    max_corr = {}
    for c in columns:
        time_shift[c] = corr_results[c].index(max(corr_results[c]))
        max_corr[c] = max(corr_results[c])
        
    return time_shift, max_corr

In [47]:
corr_sweet_spot(columns_x)

({'Index M': 10,
  'Index S': 10,
  'PMI Average': 10,
  'Building Permits': 9,
  'Consumer Sentiment': 12,
  'FED Funds Rate': 2,
  '10 Year Treasury Rate': 1,
  'Building Permits YoY': 6,
  'Consumer Sentiment YoY': 1,
  'M2 YoY': 11},
 {'Index M': 0.20095220111663836,
  'Index S': 0.3816837249333333,
  'PMI Average': 0.2970075635656211,
  'Building Permits': 0.41624672174772104,
  'Consumer Sentiment': 0.28056688536595853,
  'FED Funds Rate': 0.3769487197364978,
  '10 Year Treasury Rate': 0.15005712179142974,
  'Building Permits YoY': 0.05290466091765695,
  'Consumer Sentiment YoY': -0.16264024099981836,
  'M2 YoY': 0.471366695203756})

In [221]:
# data adjusting

col_adj = {'Index S':10,'Building Permits':9,'FED Funds Rate':2,'M2 YoY':11}
data_adj = data_shift.copy(deep=True)
data_adj = data_adj[['Index S','Building Permits','FED Funds Rate','M2 YoY','CPI Core NSA YoY']]

#reseting index for concat
data_adj['Covering Date'] = data_adj.index
data_adj.reset_index(drop=True, inplace=True)

#creating new table for 1 year forward empty space
new_rows = {'Covering Date':pd.date_range(start='2021-12-01',end='2022-12-01',freq='MS'),
            'Index S':np.nan,'Building Permits':np.nan,'FED Funds Rate':np.nan,'M2 YoY':np.nan}
new_df = pd.DataFrame(new_rows)

#concating tables together
data_final = pd.concat([data_adj,new_df],ignore_index=True)
data_final['Covering Date'] = data_final['Covering Date'].dt.strftime('%m-%Y')

#shifting values
data_final['Index S'] = data_final['Index S'].shift(col_adj['Index S'])
data_final['Building Permits'] = data_final['Building Permits'].shift(col_adj['Building Permits'])
data_final['FED Funds Rate'] = data_final['FED Funds Rate'].shift(col_adj['FED Funds Rate'])
data_final['M2 YoY'] = data_final['M2 YoY'].shift(col_adj['M2 YoY'])

#round on 2 decimal places and change columns order
data_final[['M2 YoY','CPI Core NSA YoY']] = data_final[['M2 YoY','CPI Core NSA YoY']].round(2)
data_final = data_final[['Covering Date','Index S','Building Permits','FED Funds Rate','M2 YoY','CPI Core NSA YoY']]
data_final.rename(columns={'Index S':'PMI Services'}, inplace=True)

data_final.tail(15)

,Covering Date,PMI Services,Building Permits,FED Funds Rate,M2 YoY,CPI Core NSA YoY
279,10-2021,57.2,1883.0,0.09,24.42,4.56
280,11-2021,58.7,1726.0,0.08,24.93,4.93
281,12-2021,55.3,1755.0,0.08,25.90,NaN
282,01-2022,63.7,1733.0,0.08,27.04,NaN
283,02-2022,62.7,1683.0,NaN,24.38,NaN
284,03-2022,64.0,1594.0,NaN,18.52,NaN
285,04-2022,60.1,1630.0,NaN,14.36,NaN
286,05-2022,64.1,1721.0,NaN,12.93,NaN
287,06-2022,61.7,1586.0,NaN,12.76,NaN
288,07-2022,61.9,1653.0,NaN,13.43,NaN


## Machine learning

In [222]:
data_ml = data_final.dropna()
data_ml

,Covering Date,PMI Services,Building Permits,FED Funds Rate,M2 YoY,CPI Core NSA YoY
11,06-1999,53.5,1577.0,4.74,7.01,2.08
12,07-1999,55.0,1719.0,4.74,6.76,2.08
13,08-1999,53.8,1672.0,4.76,7.39,1.90
14,09-1999,52.6,1742.0,4.99,7.90,2.01
15,10-1999,52.9,1732.0,5.07,8.22,2.06
...,...,...,...,...,...,...
276,07-2021,57.8,1595.0,0.06,23.01,4.27
277,08-2021,56.6,1696.0,0.08,23.75,4.00
278,09-2021,55.9,1758.0,0.10,23.69,4.03
279,10-2021,57.2,1883.0,0.09,24.42,4.56


In [226]:
#correlation matrix

data_ml.corr()

,PMI Services,Building Permits,FED Funds Rate,M2 YoY,CPI Core NSA YoY
PMI Services,1.000000,0.381516,0.270460,-0.164945,0.381990
Building Permits,0.381516,1.000000,0.631717,0.048302,0.416885
FED Funds Rate,0.270460,0.631717,1.000000,-0.209961,0.374799
M2 YoY,-0.164945,0.048302,-0.209961,1.000000,0.471221
CPI Core NSA YoY,0.381990,0.416885,0.374799,0.471221,1.000000


In [235]:
#creating training and testing sets

from sklearn.model_selection import train_test_split

X = data_ml[['PMI Services','Building Permits','FED Funds Rate','M2 YoY']]
y = data_ml['CPI Core NSA YoY']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.2,train_size=0.8)

print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (216, 4)
y_train shape:  (216,)
X_test shape:  (54, 4)
y_test shape:  (54,)


In [244]:
# MODEL SELECTION

#linear regression
from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(X_train,y_train)

score_train = linreg.score(X_train, y_train)
score_test = linreg.score(X_test, y_test)

print('Linear Regression model train score is {:.2f} and test score is {:.2f}'.format(score_train,score_test))

#ridge regression








Linear Regression model train score is 0.60 and test score is 0.52
